<a href="https://colab.research.google.com/github/dong-yf/100-Days-Of-ML-Code/blob/master/%E2%80%9Ctest_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
import numpy as np
import pandas as pd
import os

In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [ ]:
# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = True
num_classes = 20
subtract_pixel_mean = True
n = 3
depth = n * 6 + 2

In [ ]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_model.h5'

In [ ]:
split_rate = 0.2
def split_validation(train, split_rate):
    validation_index = int(len(train) * split_rate);
    validation = train[-validation_index:];
    train = train[:-validation_index];
    return train, validation
batch_size = 32 
num_classes = 20
epochs = 100
data_augmentation = True
#读入训练集和测试集
data = np.load("q1_data/train.npy")
x_train = data.reshape(data.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
df = pd.read_csv('q1_data/train1.csv')
y_train = df['coarse_label'].values
data = np.load("q1_data/test.npy")
x_test = data.reshape(data.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
#在训练集中分割验证集
x_train, x_validation = split_validation(x_train, split_rate)
y_train, y_validation = split_validation(y_train, split_rate)
x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_validation /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
print(x_validation.shape)
print(y_validation.shape)

(10000, 32, 32, 3)
(10000, 20)


In [ ]:
def lr_schedule(epoch):
    # Learning Rate Schedule
    lr = 0.1
    if epoch > 100:
        lr *= 0.1
    if epoch > 200:
        lr *= 0.1
    if epoch > 300:
        lr *= 0.1
    if epoch > 400:
        lr *= 0.1
    
    print('Learning rate: ', lr)
    return lr

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    # ResNet block
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet(input_shape, depth, num_classes=20):
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='softmax'))

In [ ]:
#model = resnet(input_shape=x_train.shape[1:], depth=depth)
sgd = keras.optimizers.SGD(lr=lr_schedule(0), decay=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.summary()

Learning rate:  0.1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/gdrive/My Drive/Colab Notebooks/saved_models/keras_model.h5 


In [ ]:
filepath = 'saved_models/ResNet_model.h5'
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', 
                                         histogram_freq=0, 
                                         write_graph=True, 
                                         write_images=True)
callbacks = [checkpoint, tbCallBack, lr_scheduler, lr_reducer]

In [ ]:
train_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
train_datagen.fit(x_train)

In [ ]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validation, y_validation),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs, verbose=1, workers=4,
                    callbacks=callbacks)

Learning rate:  0.1
Epoch 1/100
1249/1250 [============================>.] - ETA: 0s - loss: 2.7965 - accuracy: 0.0617
Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to saved_models/ResNet_model.h5
1250/1250 [==============================] - 25s 20ms/step - loss: 2.7964 - accuracy: 0.0616 - val_loss: 7.0060 - val_accuracy: 0.0000e+00
Learning rate:  0.1
Epoch 2/100
  86/1250 [=>............................] - ETA: 21s - loss: 2.7832 - accuracy: 0.0705